In [2]:
# Import necessary libraries
from GoogleNews import GoogleNews
import pandas as pd
import requests
from fake_useragent import UserAgent
import newspaper
from newspaper import fulltext
import re

In [3]:
# Define the keyword to search.
keyword = 'discapacidad'

In [4]:
# Perform news scraping from Google and extract the result into Pandas dataframe. 
googlenews = GoogleNews(lang='es', region='AR', period='1m', encode='utf-8')
googlenews.clear()
googlenews.search(keyword)
googlenews.get_page(1)
news_result = googlenews.result(sort=True)
news_data_df = pd.DataFrame.from_dict(news_result)

In [5]:
# Display information of dataframe.
news_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   title     20 non-null     object 
 1   media     20 non-null     object 
 2   date      20 non-null     object 
 3   datetime  0 non-null      float64
 4   desc      20 non-null     object 
 5   link      20 non-null     object 
 6   img       20 non-null     object 
dtypes: float64(1), object(6)
memory usage: 1.2+ KB


In [6]:
# Display header of dataframe.
news_data_df.head()

,title,media,date,datetime,desc,link,img
0,Corrupción en la Agencia de Discapacidad: denu...,Política Argentina,hace 45 segundos,NaN,La presentación la hizo el abogado Gregorio Da...,https://www.politicargentina.com/notas/202509/...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
1,El Gobierno promulgará la ley de emergencia en...,ADNSUR,hace 2 minutos,NaN,Después de que el Congreso derribara el veto d...,https://www.adnsur.com.ar/sociedad/el-gobierno...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
2,Ley de Discapacidad: el Gobierno nacional la p...,El Intransigente,hace 9 minutos,NaN,"Guillermo Francos confirmó su promulgación, au...",https://elintransigente.com/2025/09/ley-de-dis...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
3,"Discapacidad: promulgará la ley, pero el Gobie...",Verte,hace 10 minutos,NaN,El Gobierno no la va a reglamentar hasta defin...,https://www.verte.tv/noticias/45280-Discapacid...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
4,La Justicia ordenó restablecer las pensiones p...,Diario Hoy En la noticia,hace 14 minutos,NaN,El Juzgado Federal N° 2 hizo lugar a una medid...,https://diariohoy.net/politica/la-justicia-ord...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."


In [7]:
# ua = UserAgent()
# news_data_df_with_text = []
# for index, headers in news_data_df.iterrows():
#     news_title = str(headers['title'])
#     news_media = str(headers['media'])
#     news_update = str(headers['date'])
#     news_timestamp = str(headers['datetime'])
#     news_description = str(headers['desc'])
#     news_link = str(headers['link'])
#     print(news_link)
#     news_img = str(headers['img'])
#     try:
#         # html = requests.get(news_link).text
#         html = requests.get(news_link, headers={'User-Agent':ua.chrome}, timeout=15).text
#         text = fulltext(html)
#         print('Text Content Scraped')
#     except:
#         print('Text Content Scraped Error, Skipped')
#         pass
#     news_data_df_with_text.append([news_title, news_media, news_update, news_timestamp, 
#                                          news_description, news_link, news_img, text])

# news_data_with_text_df = pd.DataFrame(news_data_df_with_text, columns=['Title', 'Media', 'Update', 'Timestamp',
#                                                                     'Description', 'Link', 'Image', 'Text'])

from newspaper import Article

ua = UserAgent()
news_data_df_with_text = []

for index, headers in news_data_df.iterrows():
    news_title = str(headers['title'])
    news_media = str(headers['media'])
    news_update = str(headers['date'])
    news_timestamp = str(headers['datetime'])
    news_description = str(headers['desc'])
    news_link = str(headers['link'])
    news_img = str(headers['img'])

    print(news_link)

    text = ""  # initialize

    # First try Newspaper (more robust)
    try:
        article = Article(news_link, browser_user_agent=ua.chrome)
        article.download()
        article.parse()
        text = article.text
        print('Text Content via newspaper3k')
    except Exception as e:
        print(f'Newspaper extraction failed: {e}')
        # fallback to requests + fulltext
        try:
            html = requests.get(news_link, headers={'User-Agent': ua.chrome}, timeout=5).text
            text = fulltext(html)
            print('Text Content via fulltext fallback')
        except Exception as e2:
            print(f'Fallback fulltext failed: {e2}')
            # text remains empty

    news_data_df_with_text.append([
        news_title, news_media, news_update, news_timestamp,
        news_description, news_link, news_img, text
    ])

news_data_with_text_df = pd.DataFrame(
    news_data_df_with_text,
    columns=['Title', 'Media', 'Update', 'Timestamp',
             'Description', 'Link', 'Image', 'Text']
)


https://www.politicargentina.com/notas/202509/68570-corrupcion-en-la-agencia-de-discapacidad-denuncian-a-fernando-cerimedo-y-a-su-esposa-por-encubrimiento.html&ved=2ahUKEwigwbj4gtuPAxVkGbkGHSXUAs4QxfQBegQIAhAC&usg=AOvVaw3HqTkHLFXGsKmZR7RC_Jt_
Newspaper extraction failed: Article `download()` failed with 404 Client Error: Not Found for url: https://www.politicargentina.com/notas/202509/68570-corrupcion-en-la-agencia-de-discapacidad-denuncian-a-fernando-cerimedo-y-a-su-esposa-por-encubrimiento.html&ved=2ahUKEwigwbj4gtuPAxVkGbkGHSXUAs4QxfQBegQIAhAC&usg=AOvVaw3HqTkHLFXGsKmZR7RC_Jt_ on URL https://www.politicargentina.com/notas/202509/68570-corrupcion-en-la-agencia-de-discapacidad-denuncian-a-fernando-cerimedo-y-a-su-esposa-por-encubrimiento.html&ved=2ahUKEwigwbj4gtuPAxVkGbkGHSXUAs4QxfQBegQIAhAC&usg=AOvVaw3HqTkHLFXGsKmZR7RC_Jt_
Text Content via fulltext fallback
https://www.adnsur.com.ar/sociedad/el-gobierno-promulgara-la-ley-de-emergencia-en-discapacidad-pero-postergara-su-reglamentacion-p

Fallback fulltext failed: 'NoneType' object has no attribute 'xpath'


In [8]:
# Display the entire dataframe for sample checking.
news_data_with_text_df

,Title,Media,Update,Timestamp,Description,Link,Image,Text
0,Corrupción en la Agencia de Discapacidad: denu...,Política Argentina,hace 45 segundos,nan,La presentación la hizo el abogado Gregorio Da...,https://www.politicargentina.com/notas/202509/...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",Not Found\n\nThe requested URL was not found o...
1,El Gobierno promulgará la ley de emergencia en...,ADNSUR,hace 2 minutos,nan,Después de que el Congreso derribara el veto d...,https://www.adnsur.com.ar/sociedad/el-gobierno...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",
2,Ley de Discapacidad: el Gobierno nacional la p...,El Intransigente,hace 9 minutos,nan,"Guillermo Francos confirmó su promulgación, au...",https://elintransigente.com/2025/09/ley-de-dis...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",Error 404!\n\nThe page you requested does not ...
3,"Discapacidad: promulgará la ley, pero el Gobie...",Verte,hace 10 minutos,nan,El Gobierno no la va a reglamentar hasta defin...,https://www.verte.tv/noticias/45280-Discapacid...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","Salud\n\nDiscapacidad: promulgará la ley, pero..."
4,La Justicia ordenó restablecer las pensiones p...,Diario Hoy En la noticia,hace 14 minutos,nan,El Juzgado Federal N° 2 hizo lugar a una medid...,https://diariohoy.net/politica/la-justicia-ord...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",
5,A medias: Gobierno promulgará Ley de Discapaci...,ImpulsoBaires,hace 16 minutos,nan,El Poder Ejecutivo Nacional va a promulgar la ...,https://www.impulsobaires.com.ar/nota/322748/a...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","En diálogo con el diario Clarín, Guillermo Fra..."
6,La Justicia ordenó a la Andis restituir las pe...,MinutoUno,hace 17 minutos,nan,La medida tiene alcance para los afectados por...,https://www.minutouno.com/politica/la-justicia...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",
7,Fallo de los Premios cermi.es 2025,Discapnet,hace 20 minutos,nan,El jurado de los Premios cermi.,https://www.discapnet.es/node/108919&ved=2ahUK...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",
8,El Gobierno promulgará la ley de discapacidad ...,Opinando San Nicolás,hace 20 minutos,nan,El Ejecutivo confirmó que promulgará la ley de...,https://www.opinandosannicolas.ar/2025/09/el-g...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",
9,El Gobierno de Javier Milei promulgará la ley ...,Crónica,hace 22 minutos,nan,El Poder Ejecutivo había dejado trascender que...,https://www.cronica.com.ar/politica/El-Gobiern...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",


In [9]:
# Drops missiong or empty text
filtered_df = news_data_with_text_df[
    news_data_with_text_df['Text'].notna() &
    (news_data_with_text_df['Text'].str.strip() != "")
]


In [10]:
# Save the result dataframe into a CSV file.
filtered_df.to_csv("./data/news_data_with_text.csv")

In [11]:
# Reload the saved news data content from a CSV file.
news_data_with_text_df = pd.read_csv("./data/news_data_with_text.csv",  index_col=0)

In [12]:
# Select the top 10 news records (the latest 10) for summarization.
news_data_with_text_df = news_data_with_text_df.head(10)

In [13]:
# Display the scraped text content from the first news record.
print(news_data_with_text_df['Text'].values[0])

Not Found

The requested URL was not found on this server.


In [14]:
# Count the number of words in the first news record.
len(re. findall(r'\w+', news_data_with_text_df['Text'].values[0]))

11

In [15]:
# Concatenate all the top 10 news record's text content into a single string.
news_text_content_string = news_data_with_text_df.to_string(columns=['Text'], header=False, index=False)
print(news_text_content_string)

In [16]:
# Count the number of words in the top 10 news record's text content.
len(re. findall(r'\w+', news_text_content_string))

1196

In [ ]:
# Import necessary libraries
import openai
import platform
import os

import nltk
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anapa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
# Define the function to count the number of tokens.
def count_tokens(stringname):
    tokens = word_tokenize(stringname)
    return len(tokens)

In [19]:
# Display the number of tokens from the top 10 news record's text content.
stringname = news_text_content_string

token_count = count_tokens(stringname)
print(f"Number of tokens: {token_count}")

Number of tokens: 1244


In [20]:
# Define the function to divide the top 10 news record's text content into segments of 2000 tokens 
# with the overlap of 100 tokens to avoid losing information from the split.
def break_up_file(tokens, chunk_size, overlap_size):
    if len(tokens) <= chunk_size:
        yield tokens
    else:
        chunk = tokens[:chunk_size]
        yield chunk
        yield from break_up_file(tokens[chunk_size-overlap_size:], chunk_size, overlap_size)

def break_up_file_to_chunks(stringname, chunk_size=2000, overlap_size=100):
    tokens = word_tokenize(stringname)
    return list(break_up_file(tokens, chunk_size, overlap_size))

In [21]:
# Trial run of dividing function.
stringname = news_text_content_string

chunks = break_up_file_to_chunks(stringname)
for i, chunk in enumerate(chunks):
    print(f"Chunk {i}: {len(chunk)} tokens")

Chunk 0: 1244 tokens


In [22]:
# Define the function to convert tokenized text back to normal text prompts.
def convert_to_detokenized_text(tokenized_text):
    prompt_text = " ".join(tokenized_text)
    prompt_text = prompt_text.replace(" 's", "'s")
    return prompt_text

In [23]:
# Configure the baseline configuration of the OpenAI library.
openai.api_type = "azure"
openai.api_base = "https://PLESAE_ENTER_YOUR_OWNED_AOAI_RESOURCE_NAME.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = "PLEASE_ENTER_YOUR_OWNED_AOAI_SERVICE_KEY"

In [ ]:
# Perform news text content summarization by Azure OpenAI Service (GPT3) for each chunk.
# stringname = news_text_content_string

# prompt_response = []
# chunks = break_up_file_to_chunks(stringname)

# for i, chunk in enumerate(chunks):
#     print("Processing chunk " + str(i))
#     prompt_request = "Summarize this news content: " + convert_to_detokenized_text(chunks[i])
#     response = openai.Completion.create(
#             engine="eason-text-davinci-002",
#             prompt=prompt_request,
#             temperature=.5, # Default is 1.
#             max_tokens=500,
#             top_p=1 # Default is 0.5.
#     )
    
#     prompt_response.append(response["choices"][0]["text"].strip())

from openai import OpenAI

# Initialize client with API key from environment variable OPENAI_API_KEY
client = OpenAI()

stringname = news_text_content_string
prompt_response = []
chunks = break_up_file_to_chunks(stringname)

for i, chunk in enumerate(chunks):
    print("Processing chunk " + str(i))
    prompt_request = "Summarize this news content: " + convert_to_detokenized_text(chunk)

    response = client.chat.completions.create(
        model="gpt-4o-mini",  # or "gpt-3.5-turbo" if you prefer
        messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes news content."},
            {"role": "user", "content": prompt_request}
        ],
        temperature=0.5,
        max_tokens=500,
        top_p=1
    )

    prompt_response.append(response.choices[0].message.content.strip())

# Join all summaries into one string if needed
final_summary = "\n\n".join(prompt_response)
print(final_summary)


Processing chunk 0


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [25]:
# Define the prompt to perform summarization into 1,500 words for each summarized content.
prompt_request = "Consolidate these news content summaries into 1500 words sentences: " + str(prompt_response)

In [26]:
# Perform summarization by Azure OpenAI Service (GPT3) for each chunk of summarized content.
response = openai.Completion.create(
        engine="PLEASE_ENTER_YOUR_AOAI_MODEL_DEPLOYMENT_NAME",
        prompt=prompt_request,
        temperature=.5, # Default is 1.
        max_tokens=1000,
        top_p=1 # Default is 0.5.
    )

In [28]:
# Display the final summary from the top 10 news record's text content.
news_content_summary = response["choices"][0]["text"].strip()
print(news_content_summary)

The amount of text available on the internet is growing at an exponential rate. A large portion of this text is in the form of news content. As the amount of news content grows, so does the need for automated methods of summarizing this content.

One such method is to use large language models, or LLMs. LLMs are a type of generative AI that are becoming increasingly popular. These models are able to generate text, songs, poems, essays, and even pass exams. However, there is deep concern within the tech industry that these models are being developed too quickly and that they may someday run out of control and damage or destroy human society.

This concern was renewed when OpenAI, a research lab founded by student Jack Hughes, released the ChatGPT chatbot. This chatbot is able to generate responses to questions that are similar to those of a human. The release of this chatbot has caused a lot of concern within the tech industry, with many people calling for a moratorium on the developmen

In [37]:
# Select the necessary columns and convert the dataframe into HTML for showing in the email.
news_data_with_text_df_1 = news_data_with_text_df[["Title", "Media", "Timestamp", "Description", "Link"]]
news_data_with_text_df_1_html = news_data_with_text_df_1.to_html(index=False)

In [43]:
# Email the GPT news summary with the news source reference table via MailJet. 
from mailjet_rest import Client
import os
api_key = 'PLEASE_ENTER_YOUR_OWNED_MAILJET_API_KEY_NAME'
api_secret = 'PLEASE_ENTER_YOUR_OWNED_MAILJET_API_KEY_SECRET'
mailjet = Client(auth=(api_key, api_secret), version='v3.1')
data = {
  'Messages': [
    {
      "From": {
        "Email": "easonlai888@gmail.com",
        "Name": "Eason"
      },
      "To": [
        {
          "Email": "easonlai888@gmail.com",
          "Name": "Eason"
        }
      ],
      "Subject": "GPT News Summary of Today",
      "HTMLPart": "<h3>Here is the news summary of GPT for today.</h3>{}<br><br> \
                   <h3>GPT News Summary Sources</h3>{}" \
                   .format(news_content_summary, news_data_with_text_df_1_html),
    }
  ]
}
result = mailjet.send.create(data=data)

print(result.status_code)
print(result.json())


200
{'Messages': [{'Status': 'success', 'CustomID': '', 'To': [{'Email': 'easonlai888@gmail.com', 'MessageUUID': '3254b42f-5509-47d5-b473-a9bfdd8a1511', 'MessageID': 576460769578586240, 'MessageHref': 'https://api.mailjet.com/v3/REST/message/576460769578586240'}], 'Cc': [], 'Bcc': []}]}
